In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
def Conv2d(kernal, input):
    result = 0
    for i in range(3):
        for j in range(3):
            result += kernal[i][j] * input[i][j]
    if result < 0:
        result = 0
    return int(result % 256)

def Conv3d(Kernals, Inputs):
    result_s = ''
    for i in range(10):
        result = Conv2d(Kernals[i], Inputs)
        # print(result, hex(result))
        result_s += f'{result:02x}'
        if i % 4 == 3:
            result_s += '\n'
        elif i % 4 == 1:
            result_s += '_'
    return result_s

In [4]:
def load_hex_weights(file_path):
    # Read the file as a string of hex values
    with open(file_path, 'r') as f:
        data = f.read().splitlines()
    
    # Convert each space-separated hex string to integer
    weights = []
    for line in data:
        for val in line.split():
            # Handle two's complement for negative numbers in int8 range
            int_val = int(val, 16)  # Convert hex string to integer
            if int_val > 127:  # If the value is above 127, it should be a negative number
                int_val -= 256  # Convert to signed 8-bit integer (two's complement)
            weights.append(int_val)
    
    return np.array(weights, dtype=np.int8)

conv1_weight = load_hex_weights('handout_new/data/conv1_weight.txt')
conv1_weight = conv1_weight.reshape(10,3,3)
# kernal1 = [[-22,-1,16],[-10,-5,-18],[-7,17,-9]]
# kernal2 = [[3,10,-5],[3,-6,2],[-21,6,-3]]

In [5]:
def load_sample_input(file_path):
    with open(file_path, "r") as f:
        data = f.read().split()
    data = np.array(list(map(int, data)), dtype=np.int32)
    data = data.reshape(5, 1, 16, 15)
    return data

sample_input = load_sample_input('handout_new/data/sample_input.txt')

# input1 = [[36,243,199],[201,62,153],[243,140,11]]

In [24]:
for i in range(5):
    input = sample_input[0,0,i:i+3,0:3].tolist()
    print(input)
    print(Conv3d(conv1_weight, input))

[[36, 243, 199], [201, 62, 153], [243, 140, 11]]
0000_ec00
0065_00a4
5600_
[[201, 62, 153], [243, 140, 11], [7, 97, 11]]
0003_0000
00a3_a74c
54c7_
[[243, 140, 11], [7, 97, 11], [14, 205, 93]]
0090_0051
0000_8b4b
0009_
[[7, 97, 11], [14, 205, 93], [209, 121, 61]]
0000_4d53
0063_002c
a200_
[[14, 205, 93], [209, 121, 61], [139, 171, 174]]
0000_dd00
00c1_552f
0500_


In [10]:
def mac_24bit(input_patch, weight, bias=None):
    inp = input_patch.to(torch.int32)
    # print(inp,inp.shape)
    w = weight.to(torch.int32)
    # print(w,w.shape)
    acc = torch.sum(inp * w, dim=(1, 2, 3), keepdim=False)
    # print(acc)
    if bias is not None:
        acc += bias.to(torch.int32)
    #acc = torch.clamp(acc, -2**23, 2**23-1)
    #print(acc)
    # print(acc)
    acc = torch.clamp(acc, 0, 2**23-1)
    # print(acc)
    out8 = ((acc ) & 0xFF).to(torch.uint8)
    # print(out8)
    return out8

result = torch.zeros((14,13), dtype=torch.uint8)
weight = torch.from_numpy(conv1_weight[2]).unsqueeze(0).unsqueeze(0)
for row in range(14):
    for col in range(13):
        input_patch = torch.from_numpy(sample_input[0, 0, row:row+3, col:col+3]).unsqueeze(0).unsqueeze(0)
        output = mac_24bit(input_patch, weight)
        result[row, col] = output

print(result)

# input_patch = torch.from_numpy(sample_input[0, 0, 0:3, 0:3]).unsqueeze(0).unsqueeze(0)


# output = mac_24bit(input_patch, weight)
# print(output)

tensor([[236, 203,   0, 187,  29, 153,  64,   0,   0,   2,   0, 218, 236],
        [  0,   0,   0, 102,  72, 216,   0,   0,   0, 205,   0,   0,   0],
        [  0,   0, 215,  45,   0, 215,   0, 227,   0,   0, 125,  99,   0],
        [ 77,   0,  92,  18,   0,  50,   0, 121,   0, 212, 233,   0,   0],
        [221,   0,   0, 171, 191,  44,   0,   2,   0, 178,  87,   0, 185],
        [  0,   0, 178,   0,   0, 144,   0,  34,   0,   0,  12,   0,   0],
        [  3,  82,   0,   0,  90, 133,   0, 164,   0,  73,  18,   0,   0],
        [  0,  92,  18,  37,   0,   0, 170, 152,  70,  24,  22,   0, 224],
        [101, 228,   0,   5,   0, 133,   0,   0, 111, 124,  31,   0, 107],
        [ 87,   0,   0, 120,   0, 213,   0, 137,  91, 172,   0,   0,   0],
        [  0,   0, 192,  16,   0,  86,   0, 221, 140,   0,   9, 240,   0],
        [  0,   0, 107,   0,  90,   0,   0, 107,   0,   0, 156,  31,  37],
        [  0,  82, 223,   0, 161,   0, 194, 127,   0, 211, 228,   0,   9],
        [  0, 145, 222,  

In [39]:
print(weight.shape)
print(input_patch.shape)
ans = Conv2d(weight[:,0,:], input_patch[0,0,:,:])
print(ans)

torch.Size([3, 1, 3])
torch.Size([1, 1, 3, 3])
236
